In [1]:
import os
import sys
S3 = True
bucket_name = 'lrp8fruits'
path_local = 'C:/Users/leila/openclassrooms/projet8/'

In [2]:
os.getcwd()

'/home/ubuntu'

In [3]:
sys.path.append("/home/ubuntu/spark-3.0.3-bin-hadoop2.7/bin")
#sys.path.append("/home/ubuntu/spark-3.0.3-bin-hadoop2.7/python")
#sys.path.append("/home/ubuntu/spark-3.0.3-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip")


In [4]:
os.environ['SPARK_HOME'] ="/home/ubuntu/spark-3.0.3-bin-hadoop2.7"
os.environ['PYSPARK_PYTHON'] = '/home/ubuntu/testp8/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'

In [5]:
os.environ['PYSPARK_PYTHON']

'/home/ubuntu/testp8/bin/python'

In [6]:
import findspark
findspark.init()

In [7]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2,databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell'


In [8]:
import pandas as pd 
import numpy as np

import io
from PIL import Image

import tensorflow as tf

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D


In [9]:
#import clés AWS
path_keys = os.path.join("/home/ubuntu/testp8/keys.csv")

with open(path_keys,'r') as f:
        msg = f.read()
          
ID = str(msg).split('\n')[0].split(';')[1]
KEY = str(msg).split('\n')[1].split(';')[1]

# set "temporary" environment variables
os.environ["AWS_ACCESS_KEY_ID"]=ID
os.environ["AWS_SECRET_ACCESS_KEY"]=KEY
os.environ["AWS_DEFAULT_REGION"] = 'eu-west-3'
                 
                 

In [10]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, pandas_udf, PandasUDFType, split
from pyspark.ml import Pipeline
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler, PCA

Spark Context et SparkSession

In [11]:
# spark configuration
#conf = (SparkConf()
#        .set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true')
#        .set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true')
#        .setAppName('p8').setMaster('local[*]'))

conf = (SparkConf()
        .setAppName('p8').setMaster('local[*]'))

In [12]:
sc = SparkContext()
# sc.setSystemProperty('com.amazonaws.services.s3.enableV4','true')
print('modules imported')

modules imported


In [13]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [14]:
sc.setSystemProperty('com.amazonaws.services.s3.enableV4','true')
hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.access.key", ID)
hadoopConf.set("fs.s3a.secret.key", KEY)
hadoopConf.set("fs.s3a.endpoint", "s3.eu-west-3.amazonaws.com")
hadoopConf.set("fs.s3a.impl",
               "org.apache.hadoop.fs.s3a.S3AFileSystem")
# hadoopConf.set("fs.s3a.path.style.access", "true")
# hadoopConf.set("fs.s3a.connection.ssl.enabled", "true")
# hadoopConf.set(
#         "fs.s3a.aws.credentials.provider",
#        "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider",
#    )

In [15]:
spark = SparkSession(sc)

## Import data in a Spark DataFrame

In [16]:
#path = path_local+'sample/*' # pour usage local
path = "s3a://" + bucket_name + "/sample/*" #pour usgae S3

images_df = spark.read.format("binaryfile").load(path,header=True)

In [17]:
images_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



In [18]:
images_df = images_df.withColumn('label', split(col('path'),'/').getItem(4))
images_df = images_df.select('path','content','label')
images_df.show()

+--------------------+--------------------+---------+
|                path|             content|    label|
+--------------------+--------------------+---------+
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|Pineapple|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|Pineapple|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|      Fig|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|    Lemon|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|  Apricot|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|      Fig|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|    Lemon|
|s3a://lrp8fruits/...|[FF D8 FF E0 00 1...|  Apricot|
+--------------------+--------------------+---------+



## CNN ResNet50

In [ ]:
model = ResNet50(include_top=False)

In [21]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
    """
   Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
   """
    model = Sequential()
    resnet = ResNet50(weights='imagenet', include_top=False)
    resnet.set_weights(bc_model_weights.value)
    model.add(resnet)
    model.add(GlobalMaxPooling2D())
    return model

Fonctions extraites de https://docs.databricks.com/applications/machine-learning/preprocess-data/transfer-learning-tensorflow.html

In [22]:
def preprocess(content):
    """
     Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)


In [23]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

C:/Spark/spark-3.0.1-bin-hadoop2.7\python\pyspark\sql\pandas\functions.py:383: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [24]:
feat_df = images_df.select(col('path'),col('label'),featurize_udf("content").alias("features"))

In [19]:
feat_res = spark.read.option("header", 'true').csv('/home/ubuntu/feat_res2.csv')
feat_res.show()

+---+--------------------+---------+--------------------+
|_c0|                path|    label|            features|
+---+--------------------+---------+--------------------+
|  0|file:/C:/Users/33...|Pineapple|[0.0,1.8201123476...|
|  1|file:/C:/Users/33...|Pineapple|[1.16208231449127...|
|  2|file:/C:/Users/33...|      Fig|[11.6214609146118...|
|  3|file:/C:/Users/33...|    Lemon|[4.68545627593994...|
|  4|file:/C:/Users/33...|  Apricot|[5.20837974548339...|
|  5|file:/C:/Users/33...|      Fig|[9.26463317871093...|
|  6|file:/C:/Users/33...|    Lemon|[1.10543262958526...|
|  7|file:/C:/Users/33...|  Apricot|[3.24856424331665...|
+---+--------------------+---------+--------------------+



## PCA

In [20]:
list_vect = udf(lambda l: Vectors.dense(l),VectorUDT())
feat_res = feat_res.select(col('path'),col('label'),list_vect(feat_res['features']).alias("features"))


In [ ]:
scaler = StandardScaler(withMean=True, withStd=True,
                    inputCol='features',
                    outputCol = 'feats_scaled')

pca = PCA(k=7, inputCol=scaler.getOutputCol(), outputCol="pca")
pipeline = Pipeline(stages=[scaler , pca])

model_pca = pipeline.fit(feat_res)
trans_feat = model_pca.transform(feat_res)

Variance cumulee

In [ ]:
var_cum = model_pca.explainedVariance.cumsum()
sns.set_context(context='poster', font_scale=0.7)
sns.lineplot(x=[k for k in range(9)], y=np.insert(var_cum,0,0)*100, color='green')
plt.xlabel('Nb of first components')
plt.ylabel('Variance (%)')
plt.title('Variance cumulee')
plt.ylim(0,100)
plt.xlim(0,8)
plt.show()

## Export results to S3 in parquet format

In [16]:
df = spark.read.option("header", 'true').csv('/home/ubuntu/results_pca.csv')
final = df.select('path','label','pca')
final.printSchema()

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca: string (nullable = true)



In [17]:
final.show()

+--------------------+---------+--------------------+
|                path|    label|                 pca|
+--------------------+---------+--------------------+
|s3a://lrp8fruits/...|Pineapple|[32.1061333777547...|
|s3a://lrp8fruits/...|Pineapple|[30.5841415756584...|
|s3a://lrp8fruits/...|      Fig|[-25.178220340271...|
|s3a://lrp8fruits/...|    Lemon|[-2.5568563207038...|
|s3a://lrp8fruits/...|  Apricot|[-0.2432154889942...|
|s3a://lrp8fruits/...|      Fig|[-29.552612833452...|
|s3a://lrp8fruits/...|    Lemon|[-4.2925140376598...|
|s3a://lrp8fruits/...|  Apricot|[-0.8668559323320...|
+--------------------+---------+--------------------+



In [18]:
final.write.mode('overwrite').parquet("s3a://lrp8fruits/RESULTS/")